In [1]:
import os
import pandas as pd
import json

annotation_path = './data/ILT_data/ILT_data_annotations.csv'
df = pd.read_csv(annotation_path, index_col = 0)
df['img'] = df['img'].astype('str')
ucm_path = './data/UCM_captions/dataset.json'

with open(ucm_path, 'r') as f:
    contents = json.load(f)
    
contents.keys()

dict_keys(['images', 'dataset'])

In [2]:
# Get a value_count of the labels an image is assigned, and format it as [(label1, num_votes), (label2, num_votes), ...]
df2 = df.groupby('img')['label'].agg([('label_counts', lambda x: ([(v, x.value_counts()[v]) for v in x.value_counts().keys()]))]).reset_index()
# df2['more_than_one_label'] = df2['label_counts'].map(lambda x: len(x) > 1)

# Take the most voted label as the best label for the image
df2['best_label'] = df2['label_counts'].map(lambda x: max(x, key = lambda y: y[1])[0])
# Drop the label counts since we don't need them anymore
df2 = df2.drop(['label_counts'], axis = 1)
# Add the label of the image as a column to the original dataframe
df_labeled = pd.merge(df, df2, on = 'img')
display(df_labeled)

,img,folder,naam,label,caption,start,end,flag,n_seconds,best_label
0,../fotos_klein2/fotos_klein2/eigenwerken/DJI_0...,eigenwerken,Lucy,Eigenwerken/water engineering,a construction site around a river and some ye...,2023-12-18T15:13:36.577,2023-12-18T15:15:12.020,False,95,Eigenwerken/water engineering
1,../fotos_klein2/fotos_klein2/eigenwerken/DJI_0...,eigenwerken,Paul,Eigenwerken/water engineering,construction site of a lock between a canal an...,2023-12-18T16:57:22.698,2023-12-18T16:59:29.504,False,126,Eigenwerken/water engineering
2,../fotos_klein2/fotos_klein2/schepen/DJI_20230...,schepen,Lucy,Sea faring/inland shipping,some containers containing gass or stuff on a ...,2023-12-18T15:15:12.059,2023-12-18T15:16:01.399,True,49,Sea faring/inland shipping
3,../fotos_klein2/fotos_klein2/schepen/DJI_20230...,schepen,Arjan,Sea faring/inland shipping,Containers lashed to the deck of a ship,2023-12-18T15:46:04.337,2023-12-18T15:47:53.635,False,109,Sea faring/inland shipping
4,../fotos_klein2/fotos_klein2/schepen/DJI_20230...,schepen,Lucy,Sea faring/inland shipping,a ship in the harbor with open latches being f...,2023-12-18T15:16:01.432,2023-12-18T15:16:59.256,False,57,Sea faring/inland shipping
...,...,...,...,...,...,...,...,...,...,...
190,../fotos_klein2/fotos_klein2/schepen/DJI_20230...,schepen,Natalie,Sea faring/inland shipping,A zoomed in picture of the containers on a ship,2023-12-27T11:56:28.850,2023-12-27T11:56:44.581,False,15,Sea faring/inland shipping
191,../fotos_klein2/fotos_klein2/overig/DJI_0193.JPG,overig,Natalie,Miscellaneous,A grey square container next to a black square...,2023-12-27T11:58:39.398,2023-12-27T11:59:03.900,False,24,Miscellaneous
192,../fotos_klein2/fotos_klein2/overig/DJI_0148.JPG,overig,Natalie,Miscellaneous,Four pallets lying next to each other,2023-12-27T11:59:48.195,2023-12-27T12:00:07.244,False,19,Miscellaneous
193,../fotos_klein2/fotos_klein2/schepen/DJI_20230...,schepen,Natalie,Sea faring/inland shipping,A ship on a body of water in the dark surround...,2023-12-27T12:00:37.228,2023-12-27T12:01:28.923,False,51,Sea faring/inland shipping


In [52]:
# Get the value counts for the labels, per image
# NOT WORKING!!!!!!!!
df2 = df.groupby('img')['label'].agg([('label_counts', lambda x: str([[x.value_counts()[v], v] for v in x.value_counts().keys()]))]).reset_index()
df2['label_counts'] = df2['label_counts'].map(lambda x: x.split('\n'))
# Obtain which images have been labeled with more than one kind of label
df2['more_than_one_label'] = df2['label_counts'].map(lambda x: '\n' in str(x))

df_with_label_counts = pd.merge(df, df2, on = 'img')
display(df_with_label_counts)
# df['more_than_one_label'].value_counts()

,img,folder,naam,label,caption,start,end,flag,n_seconds,label_counts,more_than_one_label
0,../fotos_klein2/fotos_klein2/eigenwerken/DJI_0...,eigenwerken,Lucy,Eigenwerken/water engineering,a construction site around a river and some ye...,2023-12-18T15:13:36.577,2023-12-18T15:15:12.020,False,95,"[[[2, 'Eigenwerken/water engineering']]]",False
1,../fotos_klein2/fotos_klein2/eigenwerken/DJI_0...,eigenwerken,Paul,Eigenwerken/water engineering,construction site of a lock between a canal an...,2023-12-18T16:57:22.698,2023-12-18T16:59:29.504,False,126,"[[[2, 'Eigenwerken/water engineering']]]",False
2,../fotos_klein2/fotos_klein2/schepen/DJI_20230...,schepen,Lucy,Sea faring/inland shipping,some containers containing gass or stuff on a ...,2023-12-18T15:15:12.059,2023-12-18T15:16:01.399,True,49,"[[[2, 'Sea faring/inland shipping']]]",False
3,../fotos_klein2/fotos_klein2/schepen/DJI_20230...,schepen,Arjan,Sea faring/inland shipping,Containers lashed to the deck of a ship,2023-12-18T15:46:04.337,2023-12-18T15:47:53.635,False,109,"[[[2, 'Sea faring/inland shipping']]]",False
4,../fotos_klein2/fotos_klein2/schepen/DJI_20230...,schepen,Lucy,Sea faring/inland shipping,a ship in the harbor with open latches being f...,2023-12-18T15:16:01.432,2023-12-18T15:16:59.256,False,57,"[[[2, 'Sea faring/inland shipping']]]",False
...,...,...,...,...,...,...,...,...,...,...,...
190,../fotos_klein2/fotos_klein2/schepen/DJI_20230...,schepen,Natalie,Sea faring/inland shipping,A zoomed in picture of the containers on a ship,2023-12-27T11:56:28.850,2023-12-27T11:56:44.581,False,15,"[[[1, 'Sea faring/inland shipping']]]",False
191,../fotos_klein2/fotos_klein2/overig/DJI_0193.JPG,overig,Natalie,Miscellaneous,A grey square container next to a black square...,2023-12-27T11:58:39.398,2023-12-27T11:59:03.900,False,24,"[[[1, 'Miscellaneous']]]",False
192,../fotos_klein2/fotos_klein2/overig/DJI_0148.JPG,overig,Natalie,Miscellaneous,Four pallets lying next to each other,2023-12-27T11:59:48.195,2023-12-27T12:00:07.244,False,19,"[[[1, 'Miscellaneous']]]",False
193,../fotos_klein2/fotos_klein2/schepen/DJI_20230...,schepen,Natalie,Sea faring/inland shipping,A ship on a body of water in the dark surround...,2023-12-27T12:00:37.228,2023-12-27T12:01:28.923,False,51,"[[[1, 'Sea faring/inland shipping']]]",False


In [3]:
contents['images']

[{'sentids': [0, 1, 2, 3, 4],
  'imgid': 0,
  'sentences': [{'tokens': ['There', 'is', 'a', 'piece', 'of', 'formland'],
    'raw': 'There is a piece of farmland .',
    'imgid': 0,
    'sentid': 0},
   {'tokens': ['There', 'is', 'a', 'piece', 'of', 'cropland'],
    'raw': 'There is a piece of cropland .',
    'imgid': 0,
    'sentid': 1},
   {'tokens': ['It', 'is', 'a', 'piece', 'of', 'farmland'],
    'raw': 'It is a piece of farmland .',
    'imgid': 0,
    'sentid': 2},
   {'tokens': ['It', 'is', 'a', 'piece', 'of', 'cropland'],
    'raw': 'It is a piece of cropland .',
    'imgid': 0,
    'sentid': 3},
   {'tokens': ['Here', 'is', 'a', 'piece', 'of', 'farmland'],
    'raw': 'Here is a piece of farmland .',
    'imgid': 0,
    'sentid': 4}],
  'split': 'train',
  'filename': '1.tif'},
 {'sentids': [5, 6, 7, 8, 9],
  'imgid': 1,
  'sentences': [{'tokens': ['There', 'is', 'a', 'piece', 'of', 'formland'],
    'raw': 'There is a piece of farmland .',
    'imgid': 1,
    'sentid': 5},
   

In [14]:
ilt_dict = {'dataset': 'ILT', 'images': []}
image_paths  = df['img'].unique().tolist()
for image_path in image_paths:
    # Convert absolute image path to relative path for some flexibility
    # To do this, we remove all the folders on the path except the thematic folder + image filename
    image_path = '\\'.join(image_path.split('/')[-2:])
    print(image_path)
    image_info = {'filename': image_path}
    ilt_dict['images']
    break

eigenwerken\DJI_0060.JPG
test\eigenwerken\DJI_0060.JPG
